In [1]:
from brian2 import *
import networkx as nx
import time
# ─── 1) Simulation parameters ──────────────────────────────────────
start_scope()
N = 200                       # number of neurons
sim_duration = 5*second       # total simulation time
small_world = True
# Hodgkin–Huxley parameters (from Brian2 tutorial)
area = 20000*umetre**2
Cm   = 1*ufarad*cm**-2 * area
gL   = 0.3*msiemens*cm**-2 * area
gNa  = 120*msiemens*cm**-2 * area
gK   = 36*msiemens*cm**-2 * area
EL   = -54.387*mV
ENa  = 50*mV
EK   = -77*mV

# Glial gating parameters (Tewari & Majumdar 2012)
tau_G = 10*second  # decay time constant
alpha = 1          # jump size on each pre-spike
beta  = 0.375      # peak weight modulation (37.5%)

# Connectivity parameters
p_rand  = 0.1      # random connection probability (unused here)
k_sw    = 6        # SW neighbors
beta_sw = 0.2      # SW rewiring prob.

# Poisson input parameters
input_rate   = 100*Hz   # rate of each Poisson source
input_weight = 0.9*nA   # current injected per Poisson spike

# ─── 2) Define HH neuron equations ──────────────────────────────────
hh_eqs = '''
dv/dt = (I_L + I_Na + I_K + I_ext + I_syn) / Cm : volt
I_L    = gL * (EL - v)                       : amp
I_Na   = gNa * m**3 * h * (ENa - v)           : amp
I_K    = gK  * n**4 * (EK - v)                : amp
dm/dt  = alpha_m*(1-m) - beta_m*m             : 1
dh/dt  = alpha_h*(1-h) - beta_h*h             : 1
dn/dt  = alpha_n*(1-n) - beta_n*n             : 1

# gating rates
alpha_m = 0.1/mV * (25*mV - v) / (exp((25*mV - v)/(10*mV)) - 1)/ms : Hz
beta_m  = 4*exp(-v/(18*mV))/ms                                    : Hz
alpha_h = 0.07*exp(-v/(20*mV))/ms                                 : Hz
beta_h  = 1/(exp((30*mV - v)/(10*mV)) + 1)/ms                     : Hz
alpha_n = 0.01/mV * (10*mV - v)/(exp((10*mV - v)/(10*mV)) - 1)/ms : Hz
beta_n  = 0.125*exp(-v/(80*mV))/ms                                : Hz

I_ext : amp   # external current from PoissonGroup
I_syn : amp   # synaptic current from network
'''

neurons = NeuronGroup(
    N, model=hh_eqs,
    threshold='v > -20*mV', reset='v = EL',
    method='exponential_euler'
)
neurons.v = EL
neurons.m = 'alpha_m/(alpha_m+beta_m)'
neurons.h = 'alpha_h/(alpha_h+beta_h)'
neurons.n = 'alpha_n/(alpha_n+beta_n)'

# ─── 3) Tripartite synapse with gating variable G ──────────────────
syn_eqs = '''
dG/dt = -G/tau_G   : 1 (clock-driven)
w0 : amp            # baseline synaptic strength
'''
on_pre = '''
G += alpha
I_syn_post += w0 * (1 + beta * G)
'''

S = Synapses(neurons, neurons, model=syn_eqs, on_pre=on_pre, method='exact')
# small-world connectivity
if small_world:
    G_sw = nx.watts_strogatz_graph(N, k_sw, beta_sw)
    edge_list = list(G_sw.edges())
    i_idx = [i for i,j in edge_list] + [j for i,j in edge_list]
    j_idx = [j for i,j in edge_list] + [i for i,j in edge_list]
    S.connect(i=i_idx, j=j_idx)
else:
    S.connect(p=p_rand)
S.w0 = 0.5*nA

# ─── 4) Explicit PoissonGroup input ────────────────────────────────
poisson = PoissonGroup(N, rates=input_rate)
S_in = Synapses(poisson, neurons, on_pre='I_ext_post += input_weight')
S_in.connect(j='i')  # one-to-one mapping

# ─── 5) Monitors ───────────────────────────────────────────────────
input_mon   = SpikeMonitor(poisson)
spike_mon   = SpikeMonitor(neurons)
state_mon0  = StateMonitor(neurons, 'v', record=0)
G_monitor   = StateMonitor(S, 'G', record=[0])
I_syn_mon   = StateMonitor(neurons, 'I_syn', record=0)

# ─── 6) Run simulation ─────────────────────────────────────────────
start = time.time()
run(sim_duration)
end = time.time()
print(f"time taken in running model {end - start} s")
# Now you have:
#   input_mon.t, input_mon.i   → Poisson spike times & indices
#   spike_mon.t, spike_mon.i   → HH neuron spike times & indices
#   state_mon0.t, state_mon0.v  → voltage trace of neuron 0

# ─── 6) Basic analysis / plotting ─────────────────────────────────
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')

# raster plot
plt.figure(figsize=(6, 3))
plt.plot(spike_mon.t / ms, spike_mon.i, '.k', markersize=2)
plt.xlabel('Time (ms)')
plt.ylabel('Neuron index')
plt.title('Spike raster')
plt.tight_layout()

# membrane potential of neuron 0
plt.figure(figsize=(6, 3))
plt.plot(state_mon0.t / ms, state_mon0.v[0] / mV)
plt.xlabel('Time (ms)')
plt.ylabel('v (mV)')
plt.title('Neuron 0 membrane potential')
plt.tight_layout()
plt.show()

time taken in running model 7.860357761383057 s


In [11]:
# ─── Sanity checks ────────────────────────────────────────
# 1) Input rate
emp_input_rate = len(input_mon.t) / (N * sim_duration)
print(f"Input rate ≈ {emp_input_rate:.1f} Hz (target 100 Hz)")

# 2) Network firing rate
emp_output_rate = len(spike_mon.t) / (N * sim_duration)
print(f"Network firing rate ≈ {emp_output_rate:.1f} Hz")

# 3) Number of synapses
print(f"Total synapses: {len(S.i)} (expected ~{N*k_sw/2:.0f})")


Input rate ≈ 10000.0 Hz (target 100 Hz)
Network firing rate ≈ 9995.4 Hz
Total synapses: 1200 (expected ~600)


In [5]:
# ─── 6) Basic analysis / plotting ─────────────────────────────────
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('TkAgg')

# raster plot
plt.figure(figsize=(6, 3))
plt.plot(spike_mon.t / ms, spike_mon.i, '.k', markersize=2)
plt.xlabel('Time (ms)')
plt.ylabel('Neuron index')
plt.title('Spike raster')
plt.tight_layout()

# membrane potential of neuron 0
plt.figure(figsize=(6, 3))
plt.plot(state_mon0.t / ms, state_mon0.v[0] / mV)
plt.xlabel('Time (ms)')
plt.ylabel('v (mV)')
plt.title('Neuron 0 membrane potential')
plt.tight_layout()
plt.show()

In [4]:
# ─── Plots ───────────────────────────────────────────────────
import matplotlib.pyplot as plt

# Raster
plt.figure()
plt.plot(spike_mon.t/ms, spike_mon.i, '.k', ms=2)
plt.xlabel('Time (ms)'); plt.ylabel('Neuron index')
plt.title('Spike raster')

# Gating variable G
plt.figure()
plt.plot(G_monitor.t/second, G_monitor.G[0])
plt.xlabel('Time (s)'); plt.ylabel('G'); plt.title('Gating variable (synapse 0)')

# Postsynaptic current
plt.figure()
plt.plot(I_syn_mon.t/ms, I_syn_mon.I_syn[0]/nA)
plt.xlabel('Time (ms)'); plt.ylabel('I_syn (nA)')
plt.title('Postsynaptic current (neuron 0)')

plt.show()

In [4]:
# membrane potential of neuron 0
plt.figure(figsize=(6, 3))
plt.plot(state_mon0.t / ms, state_mon0.v[0] / mV)
plt.xlabel('Time (ms)')
plt.ylabel('v (mV)')
plt.title('Neuron 0 membrane potential')
plt.tight_layout()
plt.show()

In [3]:
import numpy as np
import matplotlib.pyplot as plt

# Suppose you’ve saved the spike times in seconds:
# input_times = np.loadtxt('input_spikes.txt')
# output_times = np.loadtxt('output_spikes.txt')

# For demonstration, let’s create dummy Poisson and network spike trains:
duration = 5.0  # seconds
bin_size = 0.005  # seconds
N = 100

# Dummy data (replace with your actual arrays):
np.random.seed(0)
# PoissonGroup: N neurons each rate 100 Hz
#input_times = np.sort(np.random.rand( int(duration * N * 100), ) * duration)
# Network: scaled Poisson output
#output_times = np.sort(np.random.rand( int(duration * N * 30), ) * duration)

input_times = input_mon.t
output_times = spike_mon.t

# Bin edges
bins = np.arange(0, duration + bin_size, bin_size)

# Count spikes per bin
input_counts, _ = np.histogram(input_times, bins=bins)
output_counts, _ = np.histogram(output_times, bins=bins)

# Convert to rates (per-neuron)
input_rate_ts = input_counts / (N * bin_size)
output_rate_ts = output_counts / (N * bin_size)

# FFT
num_bins = len(input_rate_ts)
f = np.fft.rfftfreq(num_bins, d=bin_size)
X = np.fft.rfft(input_rate_ts)
Y = np.fft.rfft(output_rate_ts)
H = (Y * np.conj(X)) / (X * np.conj(X))

# Plot
plt.figure()
plt.plot(f, np.abs(H))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain |H(f)|')
plt.title('Transfer Function Estimate')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assume you already have:
#   f   : array of frequencies (Hz) from np.fft.rfftfreq
#   H   : complex transfer function array from Sxy/Sxx
#   input_rate_ts, output_rate_ts  : your binned time series

# 1) Compute magnitude in dB (optional)
H_mag = np.abs(H)
H_db  = 20 * np.log10(H_mag / H_mag[0])   # normalize to DC gain = 0 dB

# 2) Plot gain vs frequency
plt.figure()
plt.semilogx(f, H_db, label='Network gain')
plt.axhline(-3, color='gray', ls='--', label='–3 dB')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain (dB)')
plt.title('Frequency Response (Bode plot)')
plt.legend()
plt.tight_layout()

# 3) Find –3 dB cutoff frequency
#    find the first f where H_mag drops below H_mag[0]/sqrt(2)
threshold = H_mag[0] / np.sqrt(2)
# only look beyond f>0
idx = np.where((H_mag < threshold) & (f>0))[0]
if len(idx)>0:
    fc = f[idx[0]]
    print(f"Estimated cutoff f_c = {fc:.1f} Hz (–3 dB point)")
    plt.axvline(fc, color='red', ls=':', label=f'f_c ≈ {fc:.1f} Hz')
    plt.legend()
else:
    print("No clear –3 dB point found in the FFT range")

plt.show()

In [ ]:
print(f"Neuron 0 presynaptic partners: {np.sum(S.j == 0)}")


In [ ]:
print(np.sum(S_in.j == 0))  # ⇒ 1

In [ ]:
plt.plot(Iext_mon.t/ms, Iext_mon.I_ext[0]/nA, '.-')
plt.xlabel('Time (ms)'); plt.ylabel('I_ext (nA)')
plt.show()

In [ ]:
from brian2 import *
import networkx as nx
import time
import matplotlib
matplotlib.use('TkAgg')

# ─── 1) Simulation parameters ──────────────────────────────────────
start_scope()
N = 200                      
sim_duration = 5*second      
small_world = True           

# Hodgkin–Huxley parameters
area = 20000*umetre**2
Cm   = 1*ufarad*cm**-2 * area
gL   = 0.3*msiemens*cm**-2 * area
gNa  = 120*msiemens*cm**-2 * area
gK   = 36*msiemens*cm**-2 * area
EL   = -54.387*mV
ENa  = 50*mV
EK   = -77*mV

# Glial gating parameters
tau_G = 1*second  
alpha = 1         
beta  = 0.375     

# Connectivity (small‐world)
k_sw    = 6       
beta_sw = 0.2     

# PoissonInput parameters
poisson_N      = 50       # number of independent Poisson synapses per neuron
poisson_rate   = 1000*Hz  
poisson_weight = 0.5*nA   
tau_ext        = 5*ms     

# ─── 2) Define HH neuron + decaying I_ext ─────────────────────────
hh_eqs = '''
dv/dt     = (I_L + I_Na + I_K + I_ext + I_syn) / Cm : volt
dI_ext/dt = -I_ext/tau_ext                         : amp

I_L    = gL * (EL - v)                       : amp
I_Na   = gNa * m**3 * h * (ENa - v)           : amp
I_K    = gK  * n**4 * (EK - v)                : amp

dm/dt  = alpha_m*(1-m) - beta_m*m             : 1
dh/dt  = alpha_h*(1-h) - beta_h*h             : 1
dn/dt  = alpha_n*(1-n) - beta_n*n             : 1

alpha_m = 0.1/mV * (25*mV - v) / (exp((25*mV - v)/(10*mV)) - 1)/ms : Hz
beta_m  = 4*exp(-v/(18*mV))/ms                                    : Hz
alpha_h = 0.07*exp(-v/(20*mV))/ms                                 : Hz
beta_h  = 1/(exp((30*mV - v)/(10*mV)) + 1)/ms                     : Hz
alpha_n = 0.01/mV * (10*mV - v)/(exp((10*mV - v)/(10*mV)) - 1)/ms : Hz
beta_n  = 0.125*exp(-v/(80*mV))/ms                                : Hz
I_syn : amp
'''

neurons = NeuronGroup(
    N, model=hh_eqs,
    threshold='v > -20*mV', reset='v = EL',
    method='exponential_euler'
)
neurons.v = EL
neurons.m = 'alpha_m/(alpha_m+beta_m)'
neurons.h = 'alpha_h/(alpha_h+beta_h)'
neurons.n = 'alpha_n/(alpha_n+beta_n)'

# ─── 3) Tripartite synapses ────────────────────────────────────────
syn_eqs = '''
dG/dt = -G/tau_G : 1 (clock-driven)
w0 : amp
'''
on_pre = '''
G += alpha
I_syn_post += w0 * (1 + beta * G)
'''

S = Synapses(neurons, neurons, model=syn_eqs, on_pre=on_pre, method='exact')
G_sw = nx.watts_strogatz_graph(N, k_sw, beta_sw)
edges = list(G_sw.edges())
i_idx = [i for i,j in edges] + [j for i,j in edges]
j_idx = [j for i,j in edges] + [i for i,j in edges]
S.connect(i=i_idx, j=j_idx)
S.w0 = 0.9*nA

# ─── 4) PoissonInput drives I_ext ─────────────────────────────────
PoissonInput(
    target=neurons,
    target_var='I_ext',
    N=poisson_N,
    rate=poisson_rate,
    weight=poisson_weight
)

# ─── 5) Monitors ───────────────────────────────────────────────────
spike_mon  = SpikeMonitor(neurons)
state_mon0 = StateMonitor(neurons, ['v','I_ext','I_syn'], record=0)
G_mon      = StateMonitor(S, 'G', record=[0])

# ─── 6) Run ────────────────────────────────────────────────────────
run(sim_duration)

# ─── 7) Plotting ───────────────────────────────────────────────────
import matplotlib.pyplot as plt

plt.figure()
plt.plot(state_mon0.t/ms, state_mon0.I_ext[0]/nA)
plt.xlabel('Time (ms)'); plt.ylabel('I_ext (nA)')
plt.title('External synaptic current')

plt.figure()
plt.plot(state_mon0.t/ms, state_mon0.v[0]/mV)
plt.xlabel('Time (ms)'); plt.ylabel('v (mV)')
plt.title('Neuron 0 membrane potential')

plt.figure()
plt.plot(G_mon.t/second, G_mon.G[0])
plt.xlabel('Time (s)'); plt.ylabel('G')
plt.title('Glial gating (synapse 0)')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(state_mon0.t/ms, state_mon0.I_ext[0]/nA)
plt.xlabel('Time (ms)'); plt.ylabel('I_ext (nA)')
plt.title('External synaptic current')

plt.figure()
plt.plot(state_mon0.t/ms, state_mon0.v[0]/mV)
plt.xlabel('Time (ms)'); plt.ylabel('v (mV)')
plt.title('Neuron 0 membrane potential')

plt.figure()
plt.plot(G_mon.t/second, G_mon.G[0])
plt.xlabel('Time (s)'); plt.ylabel('G')
plt.title('Glial gating (synapse 0)')

plt.tight_layout()
plt.show()
